# Dataset

[Resume Corpus](https://github.com/florex/resume_corpus)

# Setup

In [48]:
import pandas as pd
import os
import codecs
from bs4 import BeautifulSoup 
import codecs
import spacy
import anthropic
import json
import tqdm

# Extract

In [36]:
nlp = spacy.load("en_core_web_lg")

def xfm(raw_text):
    plain_text = BeautifulSoup(raw_text).text
    return plain_text

In [37]:
files = [fn for fn in os.listdir("../data/resumes_corpus") if fn[-4:] == ".lab" ]

In [39]:
recs = []
for fn in files:

    fn_prefix = fn[:-4]
    with codecs.open("../data/resumes_corpus/" + fn_prefix + ".lab", 'r', encoding='utf-8', errors='ignore') as f:
        labels = f.read()         
    labels_split = labels.split("\n")

    with codecs.open("../data/resumes_corpus/" + fn_prefix + ".txt", 'r', encoding='utf-8', errors='ignore') as f:
        raw_text = f.read()         
    plain_text = xfm(raw_text)
    
    recs.append({"id": fn_prefix, "labels": labels_split, "cv": plain_text})
    

# Transform

In [34]:
client = anthropic.Anthropic()

with open("./system.txt") as f:
    system_msg = f.read()

def entity_parsing(cv):
    try:
        message = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=4096,
            system = system_msg,
            messages=[{"role": "user", "content": cv }]
            )
        return json.loads(message.content[0].text.replace("```json","").replace("```",""))
    except Exception as e:
        return {}


In [50]:
for rec in tqdm.tqdm(recs[0:100]):
    res = entity_parsing(rec["cv"])
    res["cv"] = rec["cv"]
    res["id"] = rec["id"]
    with open("output.jsonl", "a") as f:
        f.write(json.dumps(res) + "\n")
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [14:02<00:00,  8.42s/it]
